In [423]:
'''
CREATE TABLE corona (
  num int(11) NOT NULL AUTO_INCREMENT,
  udate varchar(255) NOT NULL,
  confirmator varchar(100) NOT NULL,
  dead varchar(50) NOT NULL,
  suspected varchar(50) NOT NULL,
  inspection varchar(50) NOT NULL,
  rdate datetime DEFAULT NOW(),
  primary key (num)
) ENGINE=InnoDB AUTO_INCREMENT=4 DEFAULT CHARSET=utf8;
END'''

'\nCREATE TABLE corona (\n  num int(11) NOT NULL AUTO_INCREMENT,\n  udate varchar(255) NOT NULL,\n  confirmator varchar(100) NOT NULL,\n  dead varchar(50) NOT NULL,\n  suspected varchar(50) NOT NULL,\n  inspection varchar(50) NOT NULL,\n  rdate datetime DEFAULT NOW(),\n  primary key (num)\n) ENGINE=InnoDB AUTO_INCREMENT=4 DEFAULT CHARSET=utf8;\nEND'

In [424]:
!sudo pip3 install facebook-sdk
!sudo pip3 install pymysql

In [425]:
import os
import re
import time
import pymysql
import facebook
import requests
import configparser
import urllib.request
import pandas as pd
from PIL import Image
from bs4 import BeautifulSoup
from datetime import datetime
from google.colab import drive

In [426]:
c=drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [427]:
CNF_INI = 'drive/My Drive/development/config/config.ini'
ERR_URI = 'drive/My Drive/development/test/'
TRG_URI = 'http://coronafact.org'

In [428]:
config = configparser.ConfigParser()
config.read(CNF_INI)
HOST = config['mysql']['host']
USER = config['mysql']['user']
PASSWORD = config['mysql']['password']
DATABASE = config['mysql']['database']
PORT = config['mysql']['port']
CHARSET = config['mysql']['charset']
FISHING = config['facebook']['fishing']
TIPS = config['facebook']['tips']
LIFE = config['teams']['life']

In [429]:
def send_facebook(udate,confirmator,dead,suspected,inspection):
  datetime.today()
  today = datetime.today().strftime('%m월%d일')
  
  token = make_array(TIPS,';')
  graph = facebook.GraphAPI(token[1])
  
  message = '{}\n#앤비팁스 #Test Google Colab #신종코로나 바이러스 발생 실시간 상황\n#(출처)KMA 코로나팩트\n#마지막 업데이트:{}\n확진자:{}\n사망자:{}\n의심환자:{}\n검사중:{}'.format(today,udate,confirmator,dead,suspected,inspection)
  
  graph.put_object(token[0], 'feed', message=message)

In [430]:
def send_facebook2(udate,confirmator,dead,suspected,inspection):
  datetime.today()
  today = datetime.today().strftime('%m월%d일')
  
  token = make_array(FISHING,';')
  graph = facebook.GraphAPI(token[1])
  
  message = '{}\n#낚다 #Test Google Colab\n#신종코로나 바이러스 발생 실시간 상황\n#(출처)KMA 코로나팩트\n#마지막 업데이트:{}\n확진자:{}\n사망자:{}\n의심환자:{}\n검사중:{}'.format(today,udate,confirmator,dead,suspected,inspection)
  
  graph.put_object(token[0], 'feed', message=message)

In [431]:
def send_teams(udate,confirmator,dead,suspected,inspection):
  datetime.today()
  today = datetime.today().strftime('%m월%d일')
  
  message = '{}\n#앤비팁스 #Test Google Colab\n#(출처)KMA 코로나팩트 #마지막 업데이트:{}\n확진자:{}\n사망자:{}\n의심환자:{}\n검사중:{}'.format(today,udate,confirmator,dead,suspected,inspection)
  token = make_array(LIFE,';')
  
  headers = {'Content-Type': 'Application/JSON;charset=utf-8',}
  card = 'MessageCard'
  color = '0072C6'
  title = '신종코로나 바이러스 발생 실시간 상황'
  
  data = '{"@context":"'+token[0]+'","@type":"'+card+'","themeColor":"'+color+'","title":"'+title+'","text":"'+message+'"}'
  response = requests.post(token[1], headers=headers, data=data.encode('utf-8'))	  	

In [432]:
def make_array(string,keyword): 
  arr = []
  arr = string.split(keyword)
  return arr	

In [433]:
def make_date(string):
  numbers = re.findall('\d+', string)
  _year = numbers[0]
  _month = numbers[1]
  _day = numbers[2]
  _hour = numbers[3]
  _min = numbers[4]
  return _year+'.'+_month+'.'+_day+' '+_hour+':'+_min

In [434]:
def exec_select(conn,udate):
  with conn.cursor() as cursor:
    query = get_select(udate)
    cursor.execute(query)
    for row in cursor:
      data = row.get('cnt')
      
  return data

In [435]:
def exec_insert(conn,udate,confirmator,dead,suspected,inspection):
  try:
    query = get_insert(udate,confirmator,dead,suspected,inspection)
    with conn.cursor() as cursor:
      cursor.execute(query)
    conn.commit()
  finally:
    conn.close()

In [436]:
def get_select(udate):
  query = 'select \
    count(*) as cnt \
    from corona \
    where udate=\'{}\''.format(udate)
  return query

In [437]:
def get_insert(udate,confirmator,dead,suspected,inspection):
  query = 'insert into corona (udate,confirmator,dead,suspected,inspection) \
    values (\'{}\',\'{}\',\'{}\',\'{}\',\'{}\')'.format(udate,confirmator,dead,suspected,inspection)
  return query

In [438]:
def scrappy(soup, keyword):

  contents = soup.select('div#s2020020474239b6d6c646 main div.inside div.doz_row')
  # print(contents)

  if keyword == 'udate':
    try:
      tmp = contents[1].select('div.col-dz.col-dz-12 div#w20200204d2be26aca67e0 div._widget_data div#text_w20200204d2be26aca67e0 div.text-table div p')[1]
      tmp = tmp.select('span')[0].text.strip()
      value = make_date(tmp)
    except ValueError:
      value = 'udate'

  elif keyword == 'confirmator':
    try:
      tmp = contents[3].select('div.col-dz.col-dz-12 div#w202002044b4d320e01819 div._widget_data div.fullWidth.eight.columns1 div.projectFactsWrap1')[0]
      tmp = tmp.select('div.item.wow.fadeInUpBig.animated.animated')[0]
      data = tmp.select('p.timer.count-title.count-number.number')[0].get('data-to')
      rate = tmp.select('span')[0].text.strip()
      value = data+' '+rate
    except ValueError:
      value = 'confirmator'

  elif keyword == 'dead':
    try:
      tmp = contents[3].select('div.col-dz.col-dz-12 div#w202002044b4d320e01819 div._widget_data div.fullWidth.eight.columns1 div.projectFactsWrap1')[0]
      tmp = tmp.select('div.item.wow.fadeInUpBig.animated.animated')[1]
      data = tmp.select('p.timer.count-title.count-number.number')[0].get('data-to')
      rate = tmp.select('span')[0].text.strip()
      value = data+' '+rate
    except ValueError:
      value = 'dead'

  elif keyword == 'suspected':
    try:
      tmp = contents[3].select('div.col-dz.col-dz-12 div#w202002044b4d320e01819 div._widget_data div.fullWidth.eight.columns1 div.projectFactsWrap1')[0]
      tmp = tmp.select('div.item.wow.fadeInUpBig.animated.animated')[2]
      data = tmp.select('p.timer.count-title.count-number.number')[0].get('data-to')
      rate = tmp.select('span')[0].text.strip()
      value = data+' '+rate
    except ValueError:
      value = 'suspected' 

  elif keyword == 'inspection':
    try:
      tmp = contents[3].select('div.col-dz.col-dz-12 div#w202002044b4d320e01819 div._widget_data div.fullWidth.eight.columns1 div.projectFactsWrap1')[0]
      tmp = tmp.select('div.item.wow.fadeInUpBig.animated.animated')[3]
      data = tmp.select('p.timer.count-title.count-number.number')[0].get('data-to')
      rate = tmp.select('span')[0].text.strip()
      value = data+' '+rate
    except ValueError:
      value = 'inspection'               

  return value

In [439]:


try:
  conn = pymysql.connect(
      host=HOST,
      user=USER,
      password=PASSWORD,
      db=DATABASE,
      charset=CHARSET,
      port=int(PORT),
      cursorclass=pymysql.cursors.DictCursor)
  
  resp = requests.get(TRG_URI)

  soup = BeautifulSoup(resp.text, 'lxml')

  udate = scrappy(soup, 'udate')
  confirmator = scrappy(soup, 'confirmator')
  dead = scrappy(soup, 'dead')
  suspected = scrappy(soup, 'suspected')
  inspection = scrappy(soup, 'inspection')    

  print('udate: ', udate)
  print('confirmator: ', confirmator)
  print('dead: ', dead)
  print('suspected: ', suspected)
  print('inspection: ', inspection)
  
  cnt = exec_select(conn,udate)
  
  if cnt:
    print('overlap seq: ',cnt)
  else:
    print('does not overlap seq: ',cnt)
    exec_insert(conn,udate,confirmator,dead,suspected,inspection)
    send_facebook(udate,confirmator,dead,suspected,inspection)
    send_facebook2(udate,confirmator,dead,suspected,inspection)
    send_teams(udate,confirmator,dead,suspected,inspection)

  # send_facebook(udate,confirmator,dead,suspected,inspection)
  # send_facebook2(udate,confirmator,dead,suspected,inspection)
  # send_teams(udate,confirmator,dead,suspected,inspection)

except Exception as e:
  with open(ERR_URI+'error.log','a') as file:
    file.write('{} You got an error: {}\n'.format(datetime.today().strftime('%Y-%m-%d %H:%M:%S'),str(e)))

udate:  2020.8.10 10:12
confirmator:  14626 (▲28)
dead:  305 (▲0)
suspected:  1628303 (▲3653)
inspection:  16396 (▼407)
overlap seq:  1
